## Objectives

In this script we will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


## Imports

Import any additional libraries you may need here.


In [11]:
import glob
import pandas as pd
from datetime import datetime
import requests

### Downloading Files

In [18]:
!wget --no-check-certificate https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget --no-check-certificate https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget --no-check-certificate https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-08-10 23:12:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
  Self-signed certificate encountered.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: 'bank_market_cap_1.json.2'

     0K ..                                                    100% 8.07M=0s

2023-08-10 23:12:48 (8.07 MB/s) - 'bank_market_cap_1.json.2' saved [2815/2815]

--2023-08-10 23:12:48--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/

## Extract


### JSON Extract Function

This function will extract JSON files.


In [19]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

The following function finds JSON files and calls the function created above to extract data from them. Then store the data in a pandas dataframe

In [14]:
#Create a list with the column names
columns=['Name','Market Cap (US$ Billion)']

In [15]:
def extract():
    json_data = pd.DataFrame(columns=columns)
    data_frames_to_concat = [] # Create an empty list to store DataFrames for concatenation

    for json_file in glob.glob("*.json"):
        data = extract_from_json(json_file)
        data_frames_to_concat.append(data) # Append the extracted DataFrame to the list for concatenation
        
    # Concatenate all the DataFrames in 'data_frames_to_concat' into a single DataFrame
    json_data = pd.concat(data_frames_to_concat, ignore_index=True)
    return json_data

Then we Load the file exchange_rates.csv as a dataframe and find the exchange rate for British pounds with the symbol GBP and store it in the variable exchange_rate

In [21]:
csv_data = pd.read_csv('exchange_rates.csv', index_col = 0)
csv_data.head()

,Rates
AUD,1.297088
BGN,1.608653
BRL,5.409196
CAD,1.271426
CHF,0.886083


In [22]:
exchange_rate = csv_data.loc['GBP','Rates']
exchange_rate

0.7323984208000001

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file to find the exchange rate of USD to GBP and Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [23]:
def transform(data):
    # Write your code here
    data['Market Cap (GBP$ Billion)'] = round(data['Market Cap (US$ Billion)'] * exchange_rate,3)
    data.drop(columns = 'Market Cap (US$ Billion)', inplace = True)
    return data

## Load

This function takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`

In [164]:
def load(data):
    # Write your code here
    data.to_csv("bank_market_cap_gbp.csv")

## Logging Function


Then we Write the logging function <code>log</code> to log our data:


In [24]:
def log(message):
    timestamp_format = "%y-%h-%d %H:%M"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open ("loging file.txt", "a") as log:
        log.write(timestamp + ',' + message + '\n')

## Running the ETL Process


Loging the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [166]:
log("ETL Job Started")
log("Extract phase Started")

### Extract


Using the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [170]:
extracted_data = extract()
# Print the rows here
extracted_data.head(5)

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [171]:
log("Extract phase Ended")

### Transform


Log <code>"Transform phase Started"</code>


In [172]:
log("Transform phase Started")

Using the function <code>transform</code> and print the first 5 rows of the output

In [173]:
transform(extracted_data)
# Print the first 5 rows here
extracted_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


Log data <code>"Transform phase Ended"</code>


In [174]:
log("Transform phase Ended")

### Load


Log `"Load phase Started"`.


In [26]:
log("Load phase Started")

Calling the load function


In [177]:
load(extracted_data)

Log `"Load phase Ended"`.


In [178]:
log("Load phase Ended")

|     Made By       |
| ----------------- |
|  Muhammad Fathi   |
